In [ ]:

from skimage import io
from skimage.filters import median, gaussian
import skimage.exposure as exposure
from skimage.morphology import *
from skimage.color import rgb2gray, rgb2hsv, rgb2yuv, rgb2ycbcr
from skimage.measure import find_contours
from skimage.transform import resize
import os
import numpy as np
from commonfunctions import *
import matplotlib.pyplot as plt
from scipy.spatial import distance
from skimage.util import img_as_float
from skimage.draw import polygon
from sklearn import svm
from joblib import dump, load
import cv2
import sys
from time import sleep, time

In [ ]:
def skin_mask_rgb(img):
    red_channel = img[:,:,0]
    green_channel = img[:,:,1]
    blue_channel = img[:,:,2]
    rgb_max = np.maximum(np.maximum(red_channel, green_channel), blue_channel)
    rgb_min = np.minimum(np.minimum(red_channel, green_channel), blue_channel)
    rgb_rule_1 = np.logical_and.reduce([
        red_channel > 95, green_channel > 60, blue_channel > 60,
        rgb_max - rgb_min > 15, abs(red_channel - green_channel) < 15,red_channel > green_channel, red_channel > blue_channel
    ])
    
    rgb_rule_2 = np.logical_and.reduce([
        red_channel > 220 , green_channel > 210 , blue_channel > 170,
        abs(red_channel - green_channel) > 15, red_channel > blue_channel, green_channel > blue_channel
    ])
    return np.logical_or(rgb_rule_1, rgb_rule_2)


In [ ]:
def skin_mask_ycrcb(img):
    result = np.zeros(img.shape)
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            y = 16 + (65.481 * result[i,j,0] + 128.553 * result[i,j,1] + 24.966 * result[i,j,2])
            cg = 128 + (-81.085 * result[i,j,0] + 112 * result[i,j,1] - 30.915 * result[i,j,2])
            cr = 128 + (112 * result[i,j,0] - 93.786 * result[i,j,1] - 18.214 * result[i,j,2])
            if cg >=85 or cg <= 135:
                cmp1 = -cg+260
                cmp2 = -cg+280
                if cr >= cmp1 and cr <= cmp2:
                    result[i,j,0] = 255
                    result[i,j,1] = 255
                    result[i,j,2] = 255
    show_images([result])
    return result

In [ ]:
def merged_mask(img):
    ycbcr_mask = skin_mask_ycrcb(img)
    hsv_mask = skin_mask_rgb(img)
    rgb_mask = skin_mask_rgb(img)
    r =  np.logical_and.reduce([ycbcr_mask, hsv_mask, rgb_mask])
    return np.asarray(r, dtype=np.uint8)

In [ ]:
def hsv_mask(img):
    h,s,v = rgb2hsv(img)[:,:,0], rgb2hsv(img)[:,:,1], rgb2hsv(img)[:,:,2]
    h = np.rad2deg(h)
    hsv_mask = np.logical_or(h < 50, h > 150)
    return hsv_mask

In [ ]:
def adaptive_thresholding(img):
    hist = exposure.histogram(img, nbins=256)
    total_num_of_pixels = img.shape[0]*img.shape[1]
    initial_threshold = round(sum(hist[1]*hist[0])/total_num_of_pixels)
    grey_level_count = hist[1][-1]
    while True:
        list_of_lower_values = hist[1][hist[1] < initial_threshold]
        frequency_of_lower_values = hist[0][hist[1] < initial_threshold]
        lower_threshold = round(sum(list_of_lower_values*frequency_of_lower_values)/sum(frequency_of_lower_values))

        list_of_higher_values = hist[1][hist[1] >= initial_threshold]
        frequency_of_higher_values = hist[0][hist[1] >= initial_threshold]
        upper_threshold = round(sum(list_of_higher_values*frequency_of_higher_values)/sum(frequency_of_higher_values))
        new_threshold = round((lower_threshold + upper_threshold)/2)
        
        if new_threshold == initial_threshold:
            break
        else:
            initial_threshold = new_threshold
    return new_threshold

In [ ]:
def get_hand_contours(hand_img):
    #get the center of the hand
    hand_img = hand_img.astype(np.uint8)
    contours = find_contours(hand_img, 0.8, fully_connected='high')
    contouring_threshold = 0
    # get contoring threshold by finding the average length of the 3 largest contours
    contouring_threshold  = np.mean(sorted([len(c) for c in contours])[-3:])
    contours_saved = []
    for c in contours:
    #draw the contour if it is not too small
        if c.shape[0] > contouring_threshold: 
            #plt.plot(c[:, 1], c[:, 0], linewidth=2)
            contours_saved.append(c)
    return contours_saved

In [ ]:
def normalize_img(img):
    #resize photo so that only the hand is visible
    # get the most left pixel that is not black
    if img.shape[0] == 0 or img.shape[1] == 0:
        return img
    #check indcies are not out of bounds
    most_left = np.where(img.sum(axis=0) != 0)[0][0]
    most_top = np.where(img.sum(axis=1) != 0)[0][0]
    most_right = np.where(img.sum(axis=0) != 0)[0][-1]
    most_bottom = np.where(img.sum(axis=1) != 0)[0][-1]
    resized_img = img[most_top:most_bottom, most_left:most_right]
    return resized_img

In [ ]:
def trace_hand_contours(img, longest_contours):
    outline = np.zeros(img.shape)
    for c in longest_contours:
        #convert c to int
        c = c.astype(int)
        outline[c[:,0], c[:,1]] = 1
    return outline

In [ ]:
def fill_hand(img, contours):
    #fill the hand with white
    result = img.astype(np.uint8)
    for c in contours:
        rr, cc = polygon(c[:,0], c[:,1])
        result[rr, cc] = 1
    return result

In [ ]:
def is_bright(img):
    img = rgb2hsv(img)
    v = img[:,:,2]
    v = v*255
    v = gaussian(v, sigma=1)
    thresh = adaptive_thresholding(v)
    if thresh > 95:
        return True
    else:
        return False

In [ ]:
def resize_img(img1):
    #resize photo to 200x200
    img_new = resize(img1, (256, 256), anti_aliasing=False)
    return img_new

In [ ]:
def pre_processing (img):
    if(is_bright(img) == False):
        img = rgb2gray(img)
        img = img*255
        filterd = gaussian(img, sigma=7)
        #show_images([filterd])
        best_threshold = adaptive_thresholding(filterd)
        thresholded = filterd > best_threshold
        #show_images([thresholded])
        result = thresholded
        result = normalize_img(result)
        result = resize_img(result)
    else:
        mask = skin_mask_rgb(img)
        longest_contours = get_hand_contours(mask)
        result = trace_hand_contours(mask, longest_contours)
        result = fill_hand(result, longest_contours)
        result = normalize_img(result)
        result = resize_img(result)
    return result 

In [ ]:
def is_empty(img):
    mask = skin_mask_rgb(img)
    longest_contours = get_hand_contours(mask)
    result = trace_hand_contours(mask, longest_contours)
    result = fill_hand(result, longest_contours)
    if result.sum() == 0:
        return True
    else:
        return False

In [ ]:
def compute_gradient(image):
    gx = np.zeros((image.shape[0], image.shape[1]))
    gy = np.zeros((image.shape[0], image.shape[1]))
    image = image.astype(np.float32)
    gx[:, 1:-1] = (image[:, 2:] - image[:, :-2]) / 2
    gy[1:-1, :] = (image[2:, :] - image[:-2, :]) / 2
    
    gx[:, 0] = image[:, 1] - image[:, 0]
    gy[0, :] = image[1, :] - image[0, :]
    
    gx[:, -1] = image[:, -1] - image[:, -2]
    gy[-1, :] = image[-1, :] - image[-2, :]
    return gx, gy

In [ ]:
def hog_cell(orientations, magnitudes, n):
    bin_size = int(180 / n)
    hog = np.zeros(n)
    for i in range(orientations.shape[0]):
        for j in range(orientations.shape[1]):
            angle = orientations[i, j]
            magnitude = magnitudes[i, j]
            bin = int(angle / bin_size)
            if bin == n:
                bin = n - 1
            hog[bin] += magnitude
            
    return hog/(magnitudes.shape[0]*magnitudes.shape[1])

In [ ]:
def normalize_vector(v):
    epsion = 1e-5
    return v / np.sqrt(np.sum(v ** 2) + epsion ** 2) 

In [ ]:
def get_hog_featrue(img):
    gx, gy = compute_gradient(img)
    x, y = gx.shape
    cx , cy = 8, 8
    bx , by = 1, 1
    
    magnitude = np.sqrt(gx**2 + gy**2)
    angels = np.rad2deg(np.arctan2(gy, gx)) % 180
    
    n_cells_x = int(x / cx)
    n_cells_y = int(y / cy)
    n_blocks_x = n_cells_x - bx + 1
    n_blocks_y = n_cells_y - by + 1
    
    cells = np.zeros((n_cells_x, n_cells_y, 9))
    prev_x = 0
    for i in range(n_cells_x):
        prev_y = 0
        for j in range(n_cells_y):
            cells[i, j] = hog_cell(angels[prev_x:prev_x+cx, prev_y:prev_y+cy], magnitude[prev_x:prev_x+cx, prev_y:prev_y+cy], 9)
            prev_y += cy
        prev_x += cx
    
    cells_norm = np.zeros((n_blocks_x, n_blocks_y, 9))
    #normalize the cells
    
    for i in range(n_blocks_x):
        for j in range(n_blocks_y):
            cells_norm[i, j] = normalize_vector(cells[i:i+bx, j:j+by].ravel())
            
    return cells_norm.ravel()

In [ ]:
img = io.imread('images/hand7.jpg')
if (is_empty(img)):
    print("Empty")
r = pre_processing(img)
show_images([img, r])
#clf.predict([get_hog_featrue(r)])

In [ ]:
# define a video capture object
clf = load('model.joblib')
def flick(x):
    pass

cv2.namedWindow('image')
cv2.moveWindow('image',250,150)
cv2.namedWindow('controls')
cv2.moveWindow('controls',250,50)

controls = np.zeros((50,750),np.uint8)
cv2.putText(controls, "W/w: Play, S/s: Stay, A/a: Prev, D/d: Next, E/e: Fast, Q/q: Slow, Esc: Exit", (40,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255)

video = 'movie.mp4'
cap = cv2.VideoCapture(video)

tots = cap.get(cv2.CAP_PROP_FRAME_COUNT)
i = 0
cv2.createTrackbar('S','image', 0,int(tots)-1, flick)
cv2.setTrackbarPos('S','image',0)

cv2.createTrackbar('F','image', 1, 100, flick)
frame_rate = 30
cv2.setTrackbarPos('F','image',frame_rate)

def process(im):
    return cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

status = 'stay'
vid = cv2.VideoCapture(0)
c=0
while(True):
    c+=1
    # Capture the video frame
    #open video in full screen
    vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    ret, frame = vid.read()
    roi=frame[100:500, 100:500]
    cv2.rectangle(frame,(90,90),(500,500),(0,255,0),0)    
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    # Display the resulting frame
    cv2.imshow('frame', frame)
    #take image from the rectangle 
    cv2.imshow('roi', roi)
    #take a photo every 2 seconds and save it
    #get current time 
    if c%30==0:
        if (is_empty(roi) == False):
            result=pre_processing(roi)
            fd = get_hog_featrue(result)
            print(clf.predict([fd]))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    cv2.imshow("controls",controls)

    if i==tots-1:
      i=0
    cap.set(cv2.CAP_PROP_FRAME_COUNT, i)
    ret, im = cap.read()
    r = 750.0 / im.shape[1]
    dim = (750, int(im.shape[0] * r))
    im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)
    if im.shape[0]>600:
        im = cv2.resize(im, (500,500))
        controls = cv2.resize(controls, (im.shape[1],25))
    cv2.imshow('image', im)
    status = { ord('s'):'stay', ord('S'):'stay',
                ord('w'):'play', ord('W'):'play',
                ord('a'):'prev_frame', ord('A'):'prev_frame',
                ord('d'):'next_frame', ord('D'):'next_frame',
                ord('q'):'slow', ord('Q'):'slow',
                ord('e'):'fast', ord('E'):'fast',
                ord('c'):'snap', ord('C'):'snap',
                -1: status, 
                27: 'exit'}[cv2.waitKey(10)]
    if status == 'play':
        i = cv2.getTrackbarPos('S','image')
        cap.set(cv2.CAP_PROP_FRAME_COUNT, i)
        frame_rate = cv2.getTrackbarPos('F','image')
        sleep((0.1-frame_rate/1000.0)**21021)
        cap = cv2.VideoCapture(video)
        #set the frame number to the current frame
        #get the current frame number
        i+=1
        cv2.setTrackbarPos('S','image',i)
        continue
    if status == 'stay':
      i = cv2.getTrackbarPos('S','image')
      #freeze the frame and wait for the next command 
      # save current frame so we can play it again
      cap = cv2.VideoCapture(video)
      cap.set(cv2.CAP_PROP_FRAME_COUNT, i)
      #save the frame
      cv2.setTrackbarPos('S','image',i)
    if status == 'exit':
        break
    if status=='prev_frame':
        i-=1
        cv2.setTrackbarPos('S','image',i)
        status='stay'
    if status=='next_frame':
        i+=1
        cv2.setTrackbarPos('S','image',i)
        status='stay'
    if status=='slow':
        frame_rate = max(frame_rate - 5, 0)
        cv2.setTrackbarPos('F', 'image', frame_rate)
        status='play'
    if status=='fast':
        frame_rate = min(100,frame_rate+5)
        cv2.setTrackbarPos('F', 'image', frame_rate)
        status='play'
    if status=='snap':
        cv2.imwrite("./"+"Snap_"+str(i)+".jpg",im)
        print ("Snap of Frame",i,"Taken!")
        status='stay'
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

In [ ]:
def test_fist():
    matches = 0
    count = 0
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/own/fist/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        img = pre_processing(img)
        fd = get_hog_featrue(img)
        #print(clf.predict([fd]))
        if clf.predict([fd]) == ['fist']:
            matches += 1
        count += 1
    print(matches)
    print(count)
    return matches,count

def test_paper():
    matches = 0
    count = 0
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/own/paper/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        img = pre_processing(img)
        fd = get_hog_featrue(img)
        #print(clf.predict([fd]))
        if clf.predict([fd]) == ['paper']:
            matches += 1
        count += 1
    print(matches)
    print(count)
    return matches,count

def test_peace():
    matches = 0
    count = 0
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/own/peace/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        img = pre_processing(img)
        fd = get_hog_featrue(img)
        #print(clf.predict([fd]))
        if clf.predict([fd]) == ['peace']:
            matches += 1
        count += 1
    return matches,count

def test_perfecto():
    matches = 0
    count = 0
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/own/perfecto/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        img = pre_processing(img)
        fd = get_hog_featrue(img)
        #print(clf.predict([fd]))
        if clf.predict([fd]) == ['perfecto']:
            matches += 1
        count += 1
    return matches,count

def test_rad():
    matches = 0
    count = 0
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/own/rad/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        img = pre_processing(img)
        fd = get_hog_featrue(img)
        #print(clf.predict([fd]))
        if clf.predict([fd]) == ['rad']:
            matches += 1
        count += 1
    return matches,count

def test_thumbs():
    matches = 0
    count = 0
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/own/thumbs/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        img = pre_processing(img)
        fd = get_hog_featrue(img)
        #print(clf.predict([fd]))
        if clf.predict([fd]) == ['thumbs']:
            matches += 1
        count += 1
    return matches,count

def test_straigt():
    matches = 0
    count = 0
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/own/straight/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        img = pre_processing(img)
        fd = get_hog_featrue(img)
        if clf.predict([fd]) == ['straight']:
            matches += 1
        count += 1
    return matches,count

def test_none():
    matches = 0
    count = 0
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/own/none/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        if is_empty(img):
            matches += 1
        count += 1
    return matches,count

In [ ]:
def run_tests():
    fist_m, fist_c = test_fist()
    paper_m, paper_c = test_paper()
    peace_m, peace_c = test_peace()
    perfecto_m , perfecto_c = test_perfecto()
    rad_m, rad_c = test_rad()
    thumbs_m, thumbs_c = test_thumbs()
    straight_m , straight_c = test_straigt()
    none_m, none_c = test_none()
    print("Fist Accuracy: ",(fist_m/fist_c)*100)
    print("Paper Accuracy: ",(paper_m/paper_c)*100)
    print("Peace Accuracy: ",(peace_m/peace_c)*100)
    print("Perfecto Accuracy: ",(perfecto_m/perfecto_c)*100)
    print("Rad Accuracy: ",(rad_m/rad_c)*100)
    print("Thumbs Accuracy: ",(thumbs_m/thumbs_c)*100)
    print("Straight Accuracy: ",(straight_m/straight_c)*100)
    print("None Accuracy: ",(none_m/none_c)*100)
    print("Total Accuracy: ",((fist_m+paper_m+peace_m+perfecto_m+rad_m+thumbs_m+straight_m+none_m)/(fist_c+paper_c+peace_c+perfecto_c+rad_c+thumbs_c+straight_c+none_c))*100)    
run_tests()